In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV

"""
    k近邻算法预测用户查询业务  (惰性,不是学习的算法)
    k值取很小: 容易受异常点影响
    k值取很大: 受到样本均衡的问题
    0<x<10
    0<y<10
    1. 缩小数据范围(上课节约时间，非必要，生产不做)

"""
data = pd.read_csv("./1k/train.csv")
# 1.缩小数据范围,非必须
data = data.query("x > 1.0 & x < 1.25 & y > 2.5 & y < 2.75")



In [2]:


# # 3。将签到位置小于 n 个人的位置给删除掉
place_count = data.groupby('place_id').count()  # group_by 列为索引列
tf = place_count[place_count.row_id > 3]
# print(tf)
# print(tf.index)
tf = place_count[place_count.row_id > 3].reset_index()  # 重置索引列为数字自增
new_data = data[data['place_id'].isin(tf.place_id)]

# print(tf)
# print(tf.index)
# # 4. 数据划分
# # 所有数据特征值
data_use = new_data[["x", "y", "accuracy", "time"]]
# # 所有数据目标值
target = new_data["place_id"]
print(new_data)
print("*************************************")
print(target)
print(target.shape)
print(target.index)
print("*************************************")

# 训练集特征值    测试集特征值    训练目标值       测试目标值
practice_data, test_data, practice_target, test_target = train_test_split(data_use, target,
                                                                          test_size=0.3)  # 30% 测试集
# # 2.标准化  (训练集特征值和测试集特征值做标准化处理提高预测成功率)
# 2.1 对训练集标准化
sd = StandardScaler()
practice_data = sd.fit_transform(practice_data)
# 2.2 对测试集特征值标准化
test_data = sd.fit_transform(test_data)

# # # 5.使用 k 近邻算法
# knn = KNeighborsClassifier(n_neighbors=5)  # 默认5可不传
# #
# # print(practice_target.shape)
# # # 6.fit 训练  --> 估计器流程1
# knn.fit(practice_data, practice_target)  # 传入训练集数据,训练集目标值
# #
# # # 7.预测测试集 --> 估计器流程2
# y_predict = knn.predict(test_data)
# print("预测结果", y_predict)  # 打印预测结果
# #
# # # 8.准确率
# print("k 近邻算法的准确率为:", knn.score(test_data, test_target))

# 交叉验证: 为了让被评估的模型更加准确可信, 为了看一个参数在这个数据集当中综合表现情况
# 通常:十折交叉认证
# 超参数: 需要手动调整参数  通常是超参数搜索-网格搜索 + 交叉验证 对 k 近邻算法进行调优

knn = KNeighborsClassifier()  # 默认5可不传
# 构造超参数字典
param = {"n_neighbors": [1, 3, 5, 7, 10]}

# 测试2折,通常10折
gc = GridSearchCV(knn, param_grid=param, cv=2)
# fit 输入数据
gc.fit(practice_data, practice_target)
# 查看模型超参数调优的过程和交叉验证的结果
# print("在2折交叉验证中最好的结果:", gc.best_score_)
# print("选择最好的模型参数:", gc.best_estimator_)
# print("每次交叉验证,验证集的表现结果:", gc.cv_results_)
# print("在测试集当中的最终预测表现", gc.score(test_data, test_target))




            row_id       x       y  accuracy    time    place_id
600            600  1.2214  2.7023        17   65380  6683426742
957            957  1.1832  2.6891        58  785470  6683426742
4345          4345  1.1935  2.6550        11  400082  6889790653
4735          4735  1.1452  2.6074        49  514983  6822359752
5580          5580  1.0089  2.7287        19  732410  1527921905
6090          6090  1.1140  2.6262        11  145507  4000153867
6234          6234  1.1449  2.5003        34  316377  3741484405
8478          8478  1.2015  2.5187        72  690722  3992589015
9357          9357  1.1916  2.7323       170  319999  5163401947
12125        12125  1.1388  2.5029        69  532507  7536975002
14937        14937  1.1426  2.7441        11  445598  6780386626
20660        20660  1.2387  2.5959        65  616095  3683087833
20930        20930  1.0519  2.5208        67  163908  6399991653
21731        21731  1.2171  2.7263        99  550339  8048985799
26584        26584  1.123

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:579: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of 

在2折交叉验证中最好的结果: 0.5204357372065529
选择最好的模型参数: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')
每次交叉验证,验证集的表现结果: {'split0_test_score': array([0.44582147, 0.48149389, 0.5059454 , 0.51766873, 0.51515659]), 'split1_test_score': array([0.45137115, 0.4825413 , 0.51609607, 0.52324987, 0.52376086]), 'mean_test_score': array([0.44857288, 0.48201317, 0.51097788, 0.52043574, 0.51942239]), 'std_test_score': array([0.00277474, 0.00052369, 0.00507515, 0.00279047, 0.00430198]), 'rank_test_score': array([5, 4, 3, 1, 2], dtype=int32), 'split0_train_score': array([1.        , 0.6716062 , 0.62340317, 0.60568898, 0.58133197]), 'split1_train_score': array([1.        , 0.67442639, 0.63356222, 0.61212527, 0.5873388 ]), 'mean_train_score': array([1.        , 0.6730163 , 0.62848269, 0.60890713, 0.58433539]), 'std_train_score': array([0.        , 0.0014101 , 0.00507952, 0.00321815, 0.00300342]), 'me

In [14]:
print(type(new_data["place_id"]))
new_data["place_id"]

<class 'pandas.core.series.Series'>


600         6683426742
957         6683426742
4345        6889790653
4735        6822359752
5580        1527921905
6090        4000153867
6234        3741484405
8478        3992589015
9357        5163401947
12125       7536975002
14937       6780386626
20660       3683087833
20930       6399991653
21731       8048985799
26584       5606572086
27937       5606572086
30798       1435128522
33184       1913341282
33877       6683426742
34340       2355236719
37405       2946102544
38968       9598377925
41861       3312463746
42135       3312463746
42729       1812226671
44283       8048985799
44549       3992589015
44694       5035268417
45880       5606572086
47291       6237569496
               ...    
29068396    3741484405
29070221    2355236719
29070322    3312463746
29070934    2199223958
29071712    2946102544
29072165    5283227804
29073572    8048985799
29074121    5270522918
29077716    6683426742
29079070    1731306153
29079416    6766324666
29079931    5270522918
29083241   